In [1]:
from network_architecture_optimization.genetic_algorithms.callbacks import OnGenerationCallback
from network_architecture_optimization.genetic_algorithms.teacher import Teacher
import pygad as pg
from sklearn.datasets import load_digits



X, y = load_digits()['images'], load_digits()['target']
teacher = Teacher(X, y)
ga_instance = pg.GA(
    num_generations=10,
    initial_population=teacher.create_initial_population(),
    gene_type=int,
    num_parents_mating=4,
    fitness_func=lambda solution, solution_idx: teacher.fitness_function(solution, solution_idx),
    mutation_percent_genes=60,
    on_generation=lambda ga_instance: OnGenerationCallback().on_generation(ga_instance)
)
ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()
model = teacher.map_chromosome_to_network_architecture(solution)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 8)]            0         
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 16)                1040      
                                                                 
 dense_15 (Dense)            (None, 8)                 136       
                                                                 
 gaussian_dropout (GaussianD  (None, 8)                0     